In [1]:
import torch
import torch.nn as nn
import numpy as np
import json
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import os    
import mediapipe as mp
import torch.nn.functional as F
import cv2
import time
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [2]:
class Dataset(Dataset):
    def __init__(self, *args): #получаем n массивов данных
        cnt, labels, temp =0, [], []
        self.data = []
        for data in args: #выделяем по одному из них
            self.data.extend(torch.tensor(data).type(torch.float32)) # преобразуем каждый в тензор и добавляем к общим данным
            print(len(data))
            for j in range(len(data)):
                labels.append(cnt) #присовим номера каждому подмассиву для проверки
            cnt += 1
        self.labels = labels
            
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
    def __len__(self):
        return len(self.data)

In [3]:
data, temp = [],[]
gests = os.listdir('dataset3')
for gest in gests:
    data = []
    print(gest)
    f = open('dataset3/' + str(gest)).readline()
    f = json.loads(f)
    for i in range (len(f)):
        data.append(f[i])
    temp.append(data)
data = np.array(temp)

r_ladon_d.txt
r_kulak_d.txt
l_ladon_d2.txt
thrashhhh3.txt
l_kulak_d.txt


/var/folders/05/318hv04n4zg2jytwsxm19nsw0000gn/T/ipykernel_95053/2590571185.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array(temp)


In [4]:
dataset = Dataset(*data)
dataloader = DataLoader(dataset, shuffle=True, batch_size = 16)

2001
2001
2001
4502
2001


In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(42, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 5)
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x),negative_slope=0.001)
        x = F.leaky_relu(self.fc2(x),negative_slope=0.001)
        x = self.fc4(x)
        return x

In [6]:
net = Net()

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-5)

In [8]:
for epoch in range(30):
    running_loss = 0.0
    for samples, labels in dataloader:
        pred = net(samples)
        loss = criterion(pred, labels)
        
        loss.backward()
        optimizer.step()
        
        optimizer.zero_grad()
        running_loss += loss.item()
    print(running_loss/len(dataloader))
print('Finished Training')

0.6509328420509768
0.33493015647429947
0.26451030235184964
0.22739828261249054
0.19307303244290908
0.17444446122350024
0.15662529330421954
0.13608195738571569
0.13379378608860257
0.128684342627728
0.1143706835740272
0.11063255494802265
0.11113189099817429
0.09678918603361678
0.10106656740716113
0.09317130460846
0.08989088947329756
0.09165989423898356
0.08125491358142982
0.08446041612422613
0.08141254151955751
0.08310880028749006
0.07519923558327979
0.07666147885285444
0.07338326678862479
0.068069689679625
0.0722365702476824
0.06410991322129744
0.0651138779451236
0.07423277714744816
Finished Training


In [9]:
ls=0
for samples, labels in dataloader:
    pred = net(samples)
    v,i = torch.max(pred, 1)
    ls+=torch.sum(abs(labels - i)).item()

print((len(dataset)-ls)/len(dataset))

0.9136414521029905


In [10]:
pred = net(samples)
v,i = torch.max(pred, 1)

In [11]:
hands = mp.solutions.hands.Hands(static_image_mode=False,
                         max_num_hands=2,
                         min_tracking_confidence=0.5,
                         min_detection_confidence=0.5)

mp_pose = mp.solutions.pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

def img_text(img, text, pos):
    cv2.putText(img, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [ ]:
r_ladon_d.txt
r_kulak_d.txt
l_ladon_d2.txt
thrashhhh3.txt
l_kulak_d.txt

In [12]:
#watch video
names = ['r_ladon', 'r_kulak', 'l_ladon','thrash','l_kulak']
cap = cv2.VideoCapture(0)
cnt_iter=[]
while(True):
    _, img = cap.read()
    ch=0
    h, w, _ = img.shape
    img = cv2.flip(img, 1)
    # hands
    t = time.time()
    result = hands.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    hand_time = time.time() - t
   
    if result.multi_hand_landmarks:
        for_model = []
        for id, lm in enumerate(result.multi_hand_landmarks[0].landmark):
            cx, cy = int(lm.x*w), int(lm.y*h)
            cv2.circle(img, (cx, cy), 5, (0, 255, 0))
            for_model.append(lm.x)
            for_model.append(lm.y)
        for_model = torch.tensor([for_model]).type(torch.float)
        pred = net(for_model)
        v,i = torch.max(pred, 1)
        img_text(img, str(names[i.item()]), (100, 30))
        cnt_iter.append(i.item())
        if len(cnt_iter)==5:
            vals, counts = np.unique(cnt_iter, return_counts=True)
            #print(vals[np.argmax(counts)])
            cnt_iter=[]
            match int(vals[np.argmax(counts)]):
                case 0:
                    print('r_ladon')
                case 1:
                    print('r_kulak')
                case 2:
                    print('l_ladon')
                case 3:
                    print('thrash')
                case 4:
                    print('l_kulak')

    cv2.imshow("1",img)
    if cv2.waitKey(1)>-1:
        break
     
cv2.destroyAllWindows()

r_ladon
r_ladon
r_ladon
r_ladon
r_ladon
r_kulak
r_kulak
r_kulak
r_kulak
r_kulak
thrash
thrash
thrash
thrash
r_ladon
r_ladon
r_ladon
l_ladon
l_ladon
l_ladon
l_ladon
r_ladon
thrash
thrash
r_kulak
r_kulak
thrash
thrash
l_kulak
l_kulak
l_kulak
l_kulak
l_kulak
l_kulak
r_kulak
r_kulak
r_kulak
r_ladon
r_ladon
r_ladon
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
r_ladon
thrash
thrash
thrash
thrash
l_ladon
l_ladon
l_ladon
thrash
thrash
l_kulak
thrash
r_ladon
r_ladon
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
l_ladon
l_ladon
thrash
thrash
thrash
thrash
l_ladon
l_ladon
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
l_ladon
l_ladon
l_ladon
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
l_ladon
l_ladon
l_ladon
l_ladon
l_ladon
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thrash
thra

In [ ]:
['r_ladon', 'r_kulak', 'l_ladon','thrash','l_kulak']